In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
import csv
#1 Truy cap Nettruyen
browser = webdriver.Chrome(executable_path="D:\Code\Python_language\CrawlingProject\chromedriver.exe")
url = "http://www.nettruyen.com"
browser.get(url)
sleep(5)



In [ ]:
#2 Truy cap search
#
search_bar = browser.find_element_by_xpath("/html/body/form/header/div/div/div/div[2]/div/input")
#
search_key = input("What do you want to search?  ")
search_bar.send_keys(search_key)
#
search_bar.send_keys(Keys.ENTER)
print("Finish Searching..")

In [ ]:
#3 Lay danh sach truyen
def get_url():
    page_source = BeautifulSoup(browser.page_source,"html.parser")
    url_source_manga = page_source.find_all('a',class_='jtip')
    list_url_manga = []
    for manga in url_source_manga:
        manga_URL = manga.get('href')
        if manga_URL not in list_url_manga:
            list_url_manga.append(manga_URL)
    return list_url_manga

In [ ]:
#3.5 Lay danh sach truyen nhieu page
number_of_page =  int(input("How many page u want to search?  "))
URL_all_page = []
for page in range(number_of_page):
    URL_one_page = get_url()
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    sleep(2)
    if browser.find_element_by_class_name("next-page"):
        browser.find_element_by_class_name("next-page").click()
    else:
        break
    URL_all_page = URL_all_page + URL_one_page
    sleep(5)

In [ ]:
with open('list_manga_search_by_name.csv','w',encoding="utf-8") as f:
    headers = ["Name","Author","Status","Tag","View","URL"]
    writer = csv.DictWriter(f,delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    ######################################
    for link in URL_all_page:
        browser.get(link)
        sleep(2)
        page_source = BeautifulSoup(browser.page_source,"html.parser")
        #Get the title
        title_div = page_source.find('div',class_="center-side col-md-8")
        title_article = title_div.find('article')
        name_manga = title_article.find('h1',class_="title-detail").get_text().strip()
        #Get all info
        info_div = page_source.find('div',class_="col-xs-8 col-info")
        info_article = info_div.find('ul',class_="list-info")
        #Creat a list clean info
        list_nclean_info = []
        clean_info = []
        list_nclean_info = info_article.find_all('p',class_="col-xs-8")
        for inf in list_nclean_info:
            f = inf.get_text().strip()
            clean_info.append(f)
        #Creat result
        author_manga = clean_info[0] 
        status_manga = clean_info[1]
        tag_manga = clean_info[2]
        view_manga = clean_info[3]
        writer.writerow({headers[0]:name_manga, headers[1]:author_manga, headers[2]:status_manga, headers[3]:tag_manga, headers[4]:view_manga, headers[5]:link})
##########################################################################
sleep(5)
browser.close()  